In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e13/sample_submission.csv
/kaggle/input/playground-series-s3e13/train.csv
/kaggle/input/playground-series-s3e13/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s3e13/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e13/test.csv')

In [3]:
train.head()

,id,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,...,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash,prognosis
0,0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lyme_disease
1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tungiasis
2,2,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,Lyme_disease
3,3,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zika
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,Rift_Valley_fever


In [4]:
test.head()

,id,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,...,lymph_swells,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash
0,707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,708,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,709,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,710,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,711,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
train['prognosis'].unique()

array(['Lyme_disease', 'Tungiasis', 'Zika', 'Rift_Valley_fever',
       'West_Nile_fever', 'Malaria', 'Chikungunya', 'Plague', 'Dengue',
       'Yellow_Fever', 'Japanese_encephalitis'], dtype=object)

In [6]:
train[train['prognosis'] == 'Lyme_disease'] = 0
train[train['prognosis'] == 'Tungiasis'] = 1
train[train['prognosis'] == 'Zika'] = 2
train[train['prognosis'] == 'Rift_Valley_fever'] = 3
train[train['prognosis'] == 'West_Nile_fever'] = 4
train[train['prognosis'] == 'Malaria'] = 5
train[train['prognosis'] == 'Chikungunya'] = 6
train[train['prognosis'] == 'Plague'] = 7
train[train['prognosis'] == 'Dengue'] = 8
train[train['prognosis'] == 'Yellow_Fever'] = 9
train[train['prognosis'] == 'Japanese_encephalitis'] = 10

In [7]:
train.dtypes

id                  int64
sudden_fever      float64
headache          float64
mouth_bleed       float64
nose_bleed        float64
                   ...   
ulcers            float64
toenail_loss      float64
speech_problem    float64
bullseye_rash     float64
prognosis          object
Length: 66, dtype: object

In [8]:
train['prognosis'] = train['prognosis'].astype(str).astype(int)

In [9]:
X = train.drop('prognosis', axis = 1)

y = train['prognosis']

In [10]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(X, y)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

r_max_accuracy = 0
r_mn = 0
r_md = 0

for n in range(1, 11):
    for d in range(1, 11):        
        model = RandomForestClassifier(n_estimators = n, max_depth = d, random_state = 1)
        model.fit(train_x, train_y)
        predictions = model.predict(val_x)
        
        accuracy = 100 - (100 * (mean_absolute_error(predictions, val_y)))
        
        if accuracy > r_max_accuracy:
            r_max_accuracy = accuracy
            r_mn = n
            r_md = d

model = RandomForestClassifier(n_estimators = r_mn, max_depth = r_md + 0.5, random_state = 1)
model.fit(train_x, train_y)
predictions = model.predict(val_x)

accuracy = 100 - (100 * (mean_absolute_error(predictions, val_y)))

if accuracy > r_max_accuracy:
    r_max_accuracy = accuracy
    r_mn = n
    r_md = d

print(r_max_accuracy)
print(r_mn)
print(r_md)

100.0
1
7


In [12]:
model.fit(train_x, train_y)
predictions = model.predict(test)

train[train['prognosis'] == 0] = 'Lyme_disease'
train[train['prognosis'] == 1] = 'Tungiasis'
train[train['prognosis'] == 2] = 'Zika'
train[train['prognosis'] == 3] = 'Rift_Valley_fever'
train[train['prognosis'] == 4] = 'West_Nile_fever'
train[train['prognosis'] == 5] = 'Malaria'
train[train['prognosis'] == 6] = 'Chikungunya'
train[train['prognosis'] == 7] = 'Plague'
train[train['prognosis'] == 8] = 'Dengue'
train[train['prognosis'] == 9] = 'Yellow_Fever'
train[train['prognosis'] == 10] = 'Japanese_encephalitis'

output = pd.DataFrame({'id': test.id, 'prognosis': predictions})
output.to_csv('submission.csv', index = False)
print('Your submission was successfully saved!')

Your submission was successfully saved!
